# Wrapping a Smolagents Agent into an ACP Server

We will create another ACP agent built with Smolagents. This second agent will be a hospital server and be able to search the web to handle health-based questions for patients. Then, we will wrap the agent definition in an ACP server similarly to how we did with the first RAG agent.

## 6.1. Wrap the Agent in ACP Server

To define our agent, we will use [CodeAgent](https://smolagents.org/docs/agents-guided-tour/) from the Smolagents library. This type of agent writes and executes Python code at every step. For this agent, we will use four tools provided by Smolagents:

- CodeAgent: crucks of our agent to write up our Python function
- DuckDuckGoSearchTool: performs a web search using DuckDuckGo browser
- LiteLLMModel: forms the basic LLM that we're going to use
- VisitWebpageTool: can visit and extract content from web pages

The agent is wrapped in an ACP server using `@server.agent()` decorator. The server will also run locally using a different port number: 8001. Run the following cell to save the agent as `smolagents_server.py` under `my_acp_project`.

In [1]:
%%writefile ../my_acp_project/smolagents_server.py
# import the dependensies
from collections.abc import AsyncGenerator # it's gonna be our type for defining our agent function
from acp_sdk.models import Message, MessagePart # it structures the output
from acp_sdk.server import Context, RunYield, RunYieldResume, Server # it forms all of the ACP output together
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool # smolagents is just a type of server like insurance policy (CrewAI) 
 
# Call the server
server = Server()

# Define LLM Model
model = LiteLLMModel(
    model_id="openai/gpt-4",  
    max_tokens=2048
)

# Define the agent
@server.agent()
async def health_agent(input: list[Message], context: Context) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is a CodeAgent which supports the hospital to handle health based questions for patients. Current or prospective patients can use it to find answers about their health and hospital treatments."
    agent = CodeAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool()], model=model)
    
    # Grab the first prompt of the list and the first part from the prompt
    prompt = input[0].parts[0].content
    response = agent.run(prompt)

    # setting up the output of our ACP with MessagePart
    yield Message(parts=[MessagePart(content=str(response))])

# run this server on port 8000 because we don't want to clash with port 8001 (insurance agent server)
if __name__ == "__main__":
    server.run(port=8000)

Overwriting ../my_acp_project/smolagents_server.py


## 6.2. Run the Hospital ACP Server

Activate our configured ACP agent, we also need to run your agent from another terminal.

- Open the second terminal by running the cell below;
- Type `uv run smolagents_server.py` to run the server and activate your ACP agent.

In [2]:
# Run the server using terminal 2 because we already used terminal one for insurance agent
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/2", width=800, height=600)

Now, the hospital server acivates and works on `http://127.0.0.1:833`inside of the `uv run smolagents_server.py` directory 

We can also ignore this warning. We can move on to another steps to make this ACP server works  
`WARNING: Can not reach server, check if running on http://127.0.0.1:8333 : Request failed after 5 retries` 

**Note: How to update the `my_acp_project` locally on your machine so you can run the second server?**
- cd `my_acp_project`
- `uv add smolagents duckduckgo-search markdownify requests`

## 6.3. Resources

- [Smolagents doc](https://smolagents.org/docs/smolagent-docs/)
- [Short course: Building Code Agents with Hugging Face Smolagents ](https://www.deeplearning.ai/short-courses/building-code-agents-with-hugging-face-smolagents/)
- [Same code using a local open source model: `ollama_chat/qwen2.5:14b`](https://github.com/nicknochnack/ACPWalkthrough/blob/main/4.%20smolagents%20ACP.py)